In [1]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [51]:
import os
import pandas as pd
from utils import (
    ESTACIONES_XLSX,
    NCEP_NCAR_XLSX,
    NDEFM_XLSX,
    PREDICTIONS_FOLDER
)
from sklearn.metrics import mean_absolute_error

In [4]:
df_maximos = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1, parse_dates=['Año hid'])
df_maximos.head()

,Año hid,San Luis,Miraflores,Pozo Sarmiento,Pie de Medano,Valle Viejo,Caimancito,Casa de Piedra,Potrero del Clavillo,Piedra Grande,Los Sosa,Padilla,La Maria,San Luis Tucuman
0,1946-01-01,NaN,NaN,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947-01-01,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1948-01-01,NaN,NaN,76.3,NaN,NaN,41.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1949-01-01,NaN,NaN,64.4,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-01,NaN,NaN,60.0,NaN,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_anuales = pd.read_excel(ESTACIONES_XLSX, sheet_name=2, header=1, parse_dates=['Año hid'])
df_anuales.head()

,Año hid,San Luis,Miraflores,Pozo Sarmiento,Pie de Medano,Valle Viejo,Caimancito,Casa de Piedra,Potrero del Clavillo,Piedra Grande,Los Sosa,Padilla,La Maria,San Luis Tucuman
0,1946-01-01,NaN,NaN,745.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947-01-01,NaN,NaN,667.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1948-01-01,NaN,NaN,557.3,NaN,NaN,487.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1949-01-01,NaN,NaN,617.7,NaN,NaN,632.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-01,NaN,NaN,821.7,NaN,NaN,1236.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
target = 'San Luis Tucuman'

In [13]:
df_maximos = df_maximos[['Año hid', target]]

In [14]:
df_anuales = df_anuales[['Año hid', target]]

In [28]:
df_data_slt = pd.merge(
    df_maximos,
    df_anuales,
    how='inner',
    left_on='Año hid',
    right_on='Año hid',
    suffixes=('_max', '_anual')
).rename(
    columns={'Año hid':'ds', 'San Luis Tucuman_max':'y', 'San Luis Tucuman_anual':'anual'}
).set_index('ds')

In [33]:
df_data_slt = df_data_slt[df_data_slt.any(axis=1).cumsum().astype(bool)]

In [40]:
df_train, df_test = df_data_slt[:-10], df_data_slt[-10:]

In [41]:
mean = df_train.y.mean()

In [43]:
df_test = df_test.assign(pred=mean)

In [46]:
mean_absolute_error(df_test.y, df_test.pred)

23.330434782608695

In [52]:
df_data_slt = df_data_slt.assign(pred=df_data_slt.y.mean())
df_data_slt[-10:].reset_index(drop=True)[['pred']].to_csv(
    os.path.join(PREDICTIONS_FOLDER, 'simple_mean.csv'),
    header=False
)